In [ ]:
def process_data(raw_file, 산지공판장_file, 전국도매_file, 품목명, 거래단위, scaler=None):
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression

    # Load data
    raw_data = pd.read_csv(raw_file)
    산지공판장 = pd.read_csv(산지공판장_file)
    전국도매 = pd.read_csv(전국도매_file)

    # 타겟 및 메타데이터 필터 조건 정의
    conditions = {
    '감자': {
        'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['감자'], '품종명': ['수미'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['감자'], '품종명': ['수미']}
    },
    '건고추': {
        'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품'),
        '공판장': None, 
        '도매': None  
    },
    '깐마늘(국산)': {
        'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['마늘'], '품종명': ['깐마늘'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['마늘'], '품종명': ['깐마늘']}
    },
    '대파': {
        'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['대파'], '품종명': ['대파(일반)'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['대파'], '품종명': ['대파(일반)']}
    },
    '무': {
        'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['무'], '품종명': ['기타무'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['무'], '품종명': ['무']}
    },
    '배추': {
        'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배추'], '품종명': ['쌈배추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배추'], '품종명': ['배추']}
    },
    '사과': {
        'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['사과'], '품종명': ['후지'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['사과'], '품종명': ['후지']}
    },
    '상추': {
        'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['상추'], '품종명': ['청상추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['상추'], '품종명': ['청상추']}
    },
    '양파': {
        'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['양파'], '품종명': ['기타양파'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['양파'], '품종명': ['양파(일반)']}
    },
    '배': {
        'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배'], '품종명': ['신고'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배'], '품종명': ['신고']}
    }
    }
 
    def adjust_prices(data, conversion_factor):
        price_columns = ['평균가(원/kg)', '최저가(원/kg)', '최고가(원/kg)', '중간가(원/kg)']
        for col in price_columns:
            if col in data.columns:
                data[col] = data[col] * conversion_factor
        return data

    # Adjust prices for 산지공판장 and 전국도매
    if 품목명 in 거래단위:
        for 단위, factor in 거래단위[품목명].items():
            산지공판장 = adjust_prices(산지공판장, factor)
            전국도매 = adjust_prices(전국도매, factor)

    # Target filtering
    if 품목명 in conditions and 'target' in conditions[품목명]:
        raw_품목 = raw_data[raw_data['품목명'] == 품목명]
        filtered_data = raw_품목[conditions[품목명]['target'](raw_품목)]
    else:
        filtered_data = raw_data[raw_data['품목명'] == 품목명]

    # 필터링된 산지공판장 및 도매 데이터
    if 품목명 in conditions:
        if conditions[품목명].get('공판장'):
            for key, values in conditions[품목명]['공판장'].items():
                산지공판장 = 산지공판장[산지공판장[key].isin(values)]
        if conditions[품목명].get('도매'):
            for key, values in conditions[품목명]['도매'].items():
                전국도매 = 전국도매[전국도매[key].isin(values)]

    # Merge data
    filtered_공판장 = 산지공판장.add_prefix('공판장_').rename(columns={'공판장_시점': '시점'})
    filtered_도매 = 전국도매.add_prefix('도매_').rename(columns={'도매_시점': '시점'})
    filtered_data = filtered_data.merge(filtered_공판장, on='시점', how='left')
    filtered_data = filtered_data.merge(filtered_도매, on='시점', how='left')

    # 파생변수 생성
    if {'최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)'}.issubset(filtered_data.columns):
        filtered_data['변동폭(최고_최저)'] = filtered_data['최고가(원/kg)'] - filtered_data['최저가(원/kg)']
        filtered_data['변동폭(평균_중간)'] = filtered_data['평균가(원/kg)'] - filtered_data['중간가(원/kg)']

    # 표준편차 계산
    filtered_data['시점_날짜'] = pd.to_datetime(filtered_data['시점'], errors='coerce')
    if not filtered_data['시점_날짜'].isna().all():
        monthly_std = filtered_data.groupby(filtered_data['시점_날짜'].dt.to_period('M'))[
            ['최고가(원/kg)', '최저가(원/kg)', '평균가(원/kg)']
        ].std().add_prefix('월_표준편차_')
        filtered_data = filtered_data.merge(monthly_std, left_on=filtered_data['시점_날짜'].dt.to_period('M'), right_index=True, how='left')

    # 품목별 선형회귀분석
    if {'최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)', '평균가(원/kg)'}.issubset(filtered_data.columns):
        regression_model = LinearRegression()
        valid_rows = filtered_data.dropna(subset=['최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)', '평균가(원/kg)'])
        X = valid_rows[['최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)']]
        y = valid_rows['평균가(원/kg)']
        if not X.empty:
            regression_model.fit(X, y)
            filtered_data['월_시세'] = (
                regression_model.coef_[0] * filtered_data['최고가(원/kg)'] +
                regression_model.coef_[1] * filtered_data['중간가(원/kg)'] +
                regression_model.coef_[2] * filtered_data['최저가(원/kg)']
            )
        else:
            filtered_data['월_시세'] = np.nan

    # 데이터 타입 최적화
    numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
    filtered_data[numeric_columns] = filtered_data[numeric_columns].fillna(0).astype('float32')

    # 정규화
    if scaler is None:
        scaler = MinMaxScaler()
        filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
    else:
        filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])

    return filtered_data, scaler

In [ ]:
# def process_data(raw_file, 산지공판장_file, 전국도매_file, 품목명, 거래단위, scaler=None):
#     import pandas as pd
#     import numpy as np
#     from sklearn.preprocessing import MinMaxScaler
#     from sklearn.linear_model import LinearRegression

#     # Load data
#     raw_data = pd.read_csv(raw_file)
#     산지공판장 = pd.read_csv(산지공판장_file)
#     전국도매 = pd.read_csv(전국도매_file)

#     # 타겟 및 메타데이터 필터 조건 정의
#     conditions = {
#     '감자': {
#         'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['감자'], '품종명': ['수미'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['감자'], '품종명': ['수미']}
#     },
#     '건고추': {
#         'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품'),
#         '공판장': None, 
#         '도매': None  
#     },
#     '깐마늘(국산)': {
#         'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['마늘'], '품종명': ['깐마늘'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['마늘'], '품종명': ['깐마늘']}
#     },
#     '대파': {
#         'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['대파'], '품종명': ['대파(일반)'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['대파'], '품종명': ['대파(일반)']}
#     },
#     '무': {
#         'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['무'], '품종명': ['기타무'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['무'], '품종명': ['무']}
#     },
#     '배추': {
#         'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배추'], '품종명': ['쌈배추'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['배추'], '품종명': ['배추']}
#     },
#     '사과': {
#         'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['사과'], '품종명': ['후지'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['사과'], '품종명': ['후지']}
#     },
#     '상추': {
#         'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['상추'], '품종명': ['청상추'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['상추'], '품종명': ['청상추']}
#     },
#     '양파': {
#         'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['양파'], '품종명': ['기타양파'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['양파'], '품종명': ['양파(일반)']}
#     },
#     '배': {
#         'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
#         '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배'], '품종명': ['신고'], '등급명': ['상']},
#         '도매': {'시장명': ['*전국도매시장'], '품목명': ['배'], '품종명': ['신고']}
#     }
#     }
#     # 거래단위 조정 함수
#     def adjust_prices(data, conversion_factor):
#         price_columns = ['평균가(원/kg)', '최저가(원/kg)', '최고가(원/kg)', '중간가(원/kg)']
#         for col in price_columns:
#             if col in data.columns:
#                 data[col] = data[col] * conversion_factor
#         return data

#     # Adjust prices for 산지공판장
#     if 품목명 in 거래단위:
#         for 단위, factor in 거래단위[품목명].items():
#             산지공판장 = adjust_prices(산지공판장, factor)
#             전국도매 = adjust_prices(전국도매, factor)

#     # Target filtering
#     raw_품목 = raw_data[raw_data['품목명'] == 품목명]
#     filtered_data = raw_품목.copy()

#     # Merge 산지공판장 and 전국도매 with the raw data
#     filtered_공판장 = 산지공판장.add_prefix('공판장_').rename(columns={'공판장_시점': '시점'})
#     filtered_도매 = 전국도매.add_prefix('도매_').rename(columns={'도매_시점': '시점'})
#     filtered_data = filtered_data.merge(filtered_공판장, on='시점', how='left')
#     filtered_data = filtered_data.merge(filtered_도매, on='시점', how='left')

#     # 파생변수 생성
#     if {'최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)'}.issubset(filtered_data.columns):
#         filtered_data['변동폭(최고_최저)'] = filtered_data['최고가(원/kg)'] - filtered_data['최저가(원/kg)']
#         filtered_data['변동폭(평균_중간)'] = filtered_data['평균가(원/kg)'] - filtered_data['중간가(원/kg)']

#     # 표준편차 계산
#     filtered_data['시점_날짜'] = pd.to_datetime(filtered_data['시점'], errors='coerce')
#     if not filtered_data['시점_날짜'].isna().all():
#         monthly_std = filtered_data.groupby(filtered_data['시점_날짜'].dt.to_period('M'))[
#             ['최고가(원/kg)', '최저가(원/kg)', '평균가(원/kg)']
#         ].std().add_prefix('월_표준편차_')
#         filtered_data = filtered_data.merge(monthly_std, left_on=filtered_data['시점_날짜'].dt.to_period('M'), right_index=True, how='left')

#     # 품목별 선형회귀분석
#     if {'최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)', '평균가(원/kg)'}.issubset(filtered_data.columns):
#         regression_model = LinearRegression()
#         valid_rows = filtered_data.dropna(subset=['최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)', '평균가(원/kg)'])
#         X = valid_rows[['최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)']]
#         y = valid_rows['평균가(원/kg)']
#         if not X.empty:
#             regression_model.fit(X, y)
#             filtered_data['월_시세'] = (
#                 regression_model.coef_[0] * filtered_data['최고가(원/kg)'] +
#                 regression_model.coef_[1] * filtered_data['중간가(원/kg)'] +
#                 regression_model.coef_[2] * filtered_data['최저가(원/kg)']
#             )
#         else:
#             filtered_data['월_시세'] = np.nan

#     # Normalize numeric columns
#     numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
#     if scaler is None:
#         scaler = MinMaxScaler()
#         filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
#     else:
#         filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])

#     return filtered_data, scaler


In [ ]:
df_train = pd.read_csv('C:/Users/medici/Team_Pro/syrup/train.csv')
df_apply = pd.read_csv('C:/Users/medici/Team_Pro/syrup/TRAIN_산지공판장_2018-2021.csv')
df_market = pd.read_csv('C:/Users/medici/Team_Pro/syrup/TRAIN_전국도매_2018-2021.csv')

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
import joblib
import numpy as np
import pandas as pd
import os

# 품목별 데이터 처리 및 XGBoost 학습
품목별_predictions = {}
품목별_scalers = {}

os.makedirs('models', exist_ok=True)

# 품목 리스트와 거래단위 설정
품목_리스트 = ['건고추', '사과', '감자', '배', '깐마늘(국산)', '무', '상추', '배추', '양파', '대파']
거래단위 = {
    '감자': 20, '깐마늘(국산)': 20, '대파': 1, '무': 20,
    '배추': 10, '사과': 3, '배': 5, '상추': 0.1, '양파': 1
}

# Progress bar for 품목
pbar_outer = tqdm(품목_리스트, desc="품목 처리 중", position=0)
for 품목명 in pbar_outer:
    pbar_outer.set_description(f"품목별 전처리 및 모델 학습 -> {품목명}")
    
    # 데이터 전처리
    train_data, scaler = process_data(
        "C:/Users/medici/pybasic/train.csv",
        "C:/Users/medici/pybasic/TRAIN_산지공판장_2018-2021.csv",
        "C:/Users/medici/pybasic/TRAIN_전국도매_2018-2021.csv",
        품목명, 거래단위
    )
    품목별_scalers[품목명] = scaler
    
    # Feature/Target 분리
    features = ['최고가(원/kg)', '최저가(원/kg)', '중간가(원/kg)', '변동폭(최고_최저)', '변동폭(평균_중간)', '월_시세']
    target = '평균가격(원)'
    
    if not set(features).issubset(train_data.columns) or target not in train_data.columns:
        print(f"필수 컬럼이 누락되었습니다: {품목명}")
        continue
    
    X = train_data[features].dropna()
    y = train_data[target].loc[X.index]
    
    if X.empty or y.empty:
        print(f"데이터가 비어 있습니다: {품목명}")
        continue
    
    # Train/Test Split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # XGBoost 모델 초기화
    model = XGBRegressor(n_estimators=50, learning_rate=0.1, max_depth=3, random_state=42)
    
    # 모델 학습
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
    
    # 모델 저장
    joblib.dump(model, f'models/best_model_{품목명}.pkl')
    
    # 성능 평가
    val_predictions = model.predict(X_val)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f"{품목명} - Validation MAE: {val_mae:.4f}")
    
    # 테스트 파일 추론
    품목_predictions = []
    pbar_inner = tqdm(range(25), desc="테스트 파일 추론 중", position=1, leave=False)
    for i in pbar_inner:
        test_file = f"C:/Users/medici/pybasic/test/TEST_{i:02d}.csv"
        산지공판장_file = f"C:/Users/medici/pybasic/test/meta/TEST_산지공판장_{i:02d}.csv"
        전국도매_file = f"C:/Users/medici/pybasic/test/meta/TEST_전국도매_{i:02d}.csv"
        
        test_data, _ = process_data(test_file, 산지공판장_file, 전국도매_file, 품목명, scaler=품목별_scalers[품목명])
        test_features = test_data[features].dropna()
        
        if test_features.empty:
            print(f"테스트 데이터가 비어 있습니다: {test_file}")
            continue
        
        # 예측 수행
        predictions = model.predict(test_features)
        품목_predictions.extend(predictions)
    
    품목별_predictions[품목명] = 품목_predictions
    pbar_outer.update(1)


품목별 전처리 및 모델 학습 -> 건고추:   0%|          | 0/10 [00:15<?, ?it/s]


MemoryError: Unable to allocate 9.97 GiB for an array with shape (9, 148691282) and data type float64

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
import joblib
import os

# 타겟 조건
conditions = {
    '감자': {
        'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['감자'], '품종명': ['수미'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['감자'], '품종명': ['수미']}
    },
    '건고추': {
        'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품'),
        '공판장': None, 
        '도매': None  
    },
    '깐마늘(국산)': {
        'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['마늘'], '품종명': ['깐마늘'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['마늘'], '품종명': ['깐마늘']}
    },
    '대파': {
        'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['대파'], '품종명': ['대파(일반)'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['대파'], '품종명': ['대파(일반)']}
    },
    '무': {
        'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['무'], '품종명': ['기타무'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['무'], '품종명': ['무']}
    },
    '배추': {
        'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배추'], '품종명': ['쌈배추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배추'], '품종명': ['배추']}
    },
    '사과': {
        'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['사과'], '품종명': ['후지'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['사과'], '품종명': ['후지']}
    },
    '상추': {
        'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['상추'], '품종명': ['청상추'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['상추'], '품종명': ['청상추']}
    },
    '양파': {
        'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['양파'], '품종명': ['기타양파'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['양파'], '품종명': ['양파(일반)']}
    },
    '배': {
        'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
        '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배'], '품종명': ['신고'], '등급명': ['상']},
        '도매': {'시장명': ['*전국도매시장'], '품목명': ['배'], '품종명': ['신고']}
    }
    }
# 데이터 처리 함수
def process_data(raw_file, 산지공판장_file, 전국도매_file, 품목명, 거래단위, scaler=None):
    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression

    # Load data
    raw_data = pd.read_csv(raw_file)
    산지공판장 = pd.read_csv(산지공판장_file)
    전국도매 = pd.read_csv(전국도매_file)
 
    def adjust_prices(data, conversion_factor):
        price_columns = ['평균가(원/kg)', '최저가(원/kg)', '최고가(원/kg)', '중간가(원/kg)']
        for col in price_columns:
            if col in data.columns:
                data[col] = data[col] * conversion_factor
        return data

    # Adjust prices for 산지공판장 and 전국도매
    # if 품목명 in 거래단위:
    #     for 단위, factor in 거래단위[품목명]():
    #         산지공판장 = adjust_prices(산지공판장, factor)
    #         전국도매 = adjust_prices(전국도매, factor)
    if 품목명 in 거래단위:
        factor = 거래단위[품목명]
        산지공판장 = adjust_prices(산지공판장, factor)
        전국도매 = adjust_prices(전국도매, factor)

    # Target filtering
    if 품목명 in conditions and 'target' in conditions[품목명]:
        raw_품목 = raw_data[raw_data['품목명'] == 품목명]
        filtered_data = raw_품목[conditions[품목명]['target'](raw_품목)]
    else:
        filtered_data = raw_data[raw_data['품목명'] == 품목명]

    # 필터링된 산지공판장 및 도매 데이터
    if 품목명 in conditions:
        if conditions[품목명].get('공판장'):
            for key, values in conditions[품목명]['공판장'].items():
                산지공판장 = 산지공판장[산지공판장[key].isin(values)]
        if conditions[품목명].get('도매'):
            for key, values in conditions[품목명]['도매'].items():
                전국도매 = 전국도매[전국도매[key].isin(values)]

    # Merge data
    filtered_공판장 = 산지공판장.add_prefix('공판장_').rename(columns={'공판장_시점': '시점'})
    filtered_도매 = 전국도매.add_prefix('도매_').rename(columns={'도매_시점': '시점'})
    filtered_data = filtered_data.merge(filtered_공판장, on='시점', how='left')
    filtered_data = filtered_data.merge(filtered_도매, on='시점', how='left')

    # 파생변수 생성
    if {'최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)'}.issubset(filtered_data.columns):
        filtered_data['변동폭(최고_최저)'] = filtered_data['최고가(원/kg)'] - filtered_data['최저가(원/kg)']
        filtered_data['변동폭(평균_중간)'] = filtered_data['평균가(원/kg)'] - filtered_data['중간가(원/kg)']

    # 표준편차 계산
    filtered_data['시점_날짜'] = pd.to_datetime(filtered_data['시점'], errors='coerce')
    if not filtered_data['시점_날짜'].isna().all():
        monthly_std = filtered_data.groupby(filtered_data['시점_날짜'].dt.to_period('M'))[
            ['최고가(원/kg)', '최저가(원/kg)', '평균가(원/kg)']
        ].std().add_prefix('월_표준편차_')
        filtered_data = filtered_data.merge(monthly_std, left_on=filtered_data['시점_날짜'].dt.to_period('M'), right_index=True, how='left')

    # 품목별 선형회귀분석
    if {'최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)', '평균가(원/kg)'}.issubset(filtered_data.columns):
        regression_model = LinearRegression()
        valid_rows = filtered_data.dropna(subset=['최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)', '평균가(원/kg)'])
        X = valid_rows[['최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)']]
        y = valid_rows['평균가(원/kg)']
        if not X.empty:
            regression_model.fit(X, y)
            filtered_data['월_시세'] = (
                regression_model.coef_[0] * filtered_data['최고가(원/kg)'] +
                regression_model.coef_[1] * filtered_data['중간가(원/kg)'] +
                regression_model.coef_[2] * filtered_data['최저가(원/kg)']
            )
        else:
            filtered_data['월_시세'] = np.nan

    # 데이터 타입 최적화
    numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
    filtered_data[numeric_columns] = filtered_data[numeric_columns].fillna(0).astype('float32')

    # 정규화
    if scaler is None:
        scaler = MinMaxScaler()
        filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
    else:
        filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])

    return filtered_data, scaler

# 학습 및 추론
품목_리스트 = ['감자', '건고추', '깐마늘(국산)', '대파', '무', '배추', '사과', '상추', '양파', '배']
거래단위 = {'감자': 20, '깐마늘': 20, '대파': 1, '무': 20, '배추': 10, '사과': 3, '배': 5, '상추': 0.1, '양파': 1}

os.makedirs('models', exist_ok=True)
os.makedirs('predictions', exist_ok=True)

품목별_predictions = {}
품목별_scalers = {}

pbar_outer = tqdm(품목_리스트, desc="품목 처리 중")
for 품목명 in pbar_outer:
    # 데이터 전처리
    train_data, scaler = process_data(
        "C:/Users/medici/pybasic/train.csv",
        "C:/Users/medici/pybasic/TRAIN_산지공판장_2018-2021.csv",
        "C:/Users/medici/pybasic/TRAIN_전국도매_2018-2021.csv",
        품목명, 거래단위)
    품목별_scalers[품목명] = scaler

    # Feature/Target 설정
    features = train_data.drop(columns=['평균가격(원)', '시점']).columns
    target = '평균가격(원)'

    X = train_data[features].values
    y = train_data[target].values

    # Train/Test Split
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 학습
    model = XGBRegressor(n_estimators=50, learning_rate=0.1, max_depth=3, random_state=42)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)

    # 모델 저장
    joblib.dump(model, f'models/best_model_{품목명}.pkl')

    # 테스트 데이터 추론
    predictions = []
    for i in range(25):  # 테스트 데이터셋 처리
        test_file = f"C:/Users/medici/pybasic/test/TEST_{i:02d}.csv"
        산지공판장_file = f"C:/Users/medici/pybasic/test/meta/TEST_산지공판장_{i:02d}.csv"
        전국도매_file = f"C:/Users/medici/pybasic/test/meta/TEST_전국도매_{i:02d}.csv"

        test_data, _ = process_data(test_file, 산지공판장_file, 전국도매_file, 품목명, scaler=scaler)
        test_features = test_data[features].values

        # 예측
        predictions.extend(model.predict(test_features))

    품목별_predictions[품목명] = predictions

# 결과 저장
joblib.dump(품목별_predictions, 'predictions/품목별_predictions.pkl')


품목 처리 중:   0%|          | 0/10 [00:00<?, ?it/s]C:\Users\medici\AppData\Local\Temp\ipykernel_25852\523808611.py:123: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered_data['시점_날짜'] = pd.to_datetime(filtered_data['시점'], errors='coerce')
품목 처리 중:   0%|          | 0/10 [00:00<?, ?it/s]


ValueError: could not convert string to float: '감자'

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from tqdm import tqdm
import joblib
import os


# 1. 데이터 로드 함수
def load_data(raw_file, 산지공판장_file, 전국도매_file):
    return pd.read_csv(raw_file), pd.read_csv(산지공판장_file), pd.read_csv(전국도매_file)


# 2. 가격 조정 함수
def adjust_prices(data, conversion_factor):
    price_columns = ['평균가(원/kg)', '최저가(원/kg)', '최고가(원/kg)', '중간가(원/kg)']
    for col in price_columns:
        if col in data.columns:
            data[col] = data[col] * conversion_factor
    return data


# 3. 타겟 필터링 함수
def filter_target_data(raw_data, 품목명, conditions):
    if 품목명 in conditions and 'target' in conditions[품목명]:
        raw_품목 = raw_data[raw_data['품목명'] == 품목명]
        return raw_품목[conditions[품목명]['target'](raw_품목)]
    return raw_data[raw_data['품목명'] == 품목명]


# 4. 메타데이터 필터링 함수
def filter_metadata_data(산지공판장, 전국도매, 품목명, conditions):
    if 품목명 in conditions:
        if conditions[품목명].get('공판장'):
            for key, values in conditions[품목명]['공판장'].items():
                산지공판장 = 산지공판장[산지공판장[key].isin(values)]
        if conditions[품목명].get('도매'):
            for key, values in conditions[품목명]['도매'].items():
                전국도매 = 전국도매[전국도매[key].isin(values)]
    return 산지공판장, 전국도매


# 5. 데이터 병합 함수
def merge_data(filtered_data, 산지공판장, 전국도매):
    try:
        filtered_공판장 = 산지공판장.add_prefix('공판장_').rename(columns={'공판장_시점': '시점'})
        filtered_도매 = 전국도매.add_prefix('도매_').rename(columns={'도매_시점': '시점'})
        filtered_data = filtered_data.merge(filtered_공판장, on='시점', how='left')
        filtered_data = filtered_data.merge(filtered_도매, on='시점', how='left')
    except KeyError as e:
        print(f"병합 중 오류 발생: {e}")
    return filtered_data


# 6. 파생 변수 생성 함수
def generate_features(filtered_data):
    if {'최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)'}.issubset(filtered_data.columns):
        filtered_data['변동폭(최고_최저)'] = filtered_data['최고가(원/kg)'] - filtered_data['최저가(원/kg)']
        filtered_data['변동폭(평균_중간)'] = filtered_data['평균가(원/kg)'] - filtered_data['중간가(원/kg)']

    filtered_data['시점_날짜'] = pd.to_datetime(filtered_data['시점'], errors='coerce')
    if not filtered_data['시점_날짜'].isna().all():
        monthly_std = filtered_data.groupby(filtered_data['시점_날짜'].dt.to_period('M'))[
            ['최고가(원/kg)', '최저가(원/kg)', '평균가(원/kg)']
        ].std().add_prefix('월_표준편차_')
        filtered_data = filtered_data.merge(monthly_std, left_on=filtered_data['시점_날짜'].dt.to_period('M'), right_index=True, how='left')

    return filtered_data


# 7. 선형 회귀 분석 함수
def add_regression_feature(filtered_data):
    if {'최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)', '평균가(원/kg)'}.issubset(filtered_data.columns):
        regression_model = LinearRegression()
        valid_rows = filtered_data.dropna(subset=['최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)', '평균가(원/kg)'])
        X = valid_rows[['최고가(원/kg)', '중간가(원/kg)', '최저가(원/kg)']]
        y = valid_rows['평균가(원/kg)']
        if not X.empty:
            regression_model.fit(X, y)
            filtered_data['월_시세'] = (
                regression_model.coef_[0] * filtered_data['최고가(원/kg)'] +
                regression_model.coef_[1] * filtered_data['중간가(원/kg)'] +
                regression_model.coef_[2] * filtered_data['최저가(원/kg)']
            )
        else:
            filtered_data['월_시세'] = np.nan
    return filtered_data


# 8. 데이터 정규화 함수
def preprocess_data(filtered_data, scaler=None):
    numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns
    filtered_data[numeric_columns] = filtered_data[numeric_columns].fillna(0).astype('float32')

    if scaler is None:
        scaler = MinMaxScaler()
        filtered_data[numeric_columns] = scaler.fit_transform(filtered_data[numeric_columns])
    else:
        filtered_data[numeric_columns] = scaler.transform(filtered_data[numeric_columns])

    return filtered_data, scaler


# 9. 모델 학습 함수
def train_model(X_train, y_train, X_val, y_val):
    """그리드 서치를 통한 모델 학습."""
    model = XGBRegressor(tree_method='gpu_hist', predictor='gpu_predictor', random_state=42)

    param_grid = {
        'n_estimators': range(100, 1100, 200),  # [100, 300, 500, 700, 900]
        'learning_rate': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2],  # 매우 낮은 학습률부터 높은 학습률까지
        'max_depth': range(3, 11),  # [3, 4, 5, ..., 10] (트리 깊이: 얕은 트리부터 깊은 트리까지)
        'subsample': [i / 10.0 for i in range(5, 11)],  # [0.5, 0.6, ..., 1.0]
        'colsample_bytree': [i / 10.0 for i in range(5, 11)]  # [0.5, 0.6, ..., 1.0]
    }

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=3,
        verbose=2,
        n_jobs=-1,
        error_score='raise'  # 정확한 오류 추적
    )
    # 그리드서치 실행
    grid_search.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)

    # 최적의 하이퍼파라미터와 성능 출력
    print(f"최적의 하이퍼파라미터: {grid_search.best_params_}")
    print(f"최적의 성능: {-grid_search.best_score_:.4f}")

    # 최적의 모델 반환
    best_model = grid_search.best_estimator_
    return best_model


# 10. 전체 워크플로 함수
def process_workflow(품목_리스트, 거래단위, conditions, train_files, test_files, output_file):
    os.makedirs('models', exist_ok=True)

    품목별_predictions = {}
    pbar_outer = tqdm(품목_리스트, desc="품목 처리 중")
    for 품목명 in pbar_outer:
        try:
            raw_data, 산지공판장, 전국도매 = load_data(*train_files)

            if 품목명 in 거래단위:
                factor = 거래단위[품목명]
                산지공판장 = adjust_prices(산지공판장, factor)
                전국도매 = adjust_prices(전국도매, factor)

            filtered_data = filter_target_data(raw_data, 품목명, conditions)
            산지공판장, 전국도매 = filter_metadata_data(산지공판장, 전국도매, 품목명, conditions)
            filtered_data = merge_data(filtered_data, 산지공판장, 전국도매)

            filtered_data = generate_features(filtered_data)
            filtered_data = add_regression_feature(filtered_data)
            filtered_data, scaler = preprocess_data(filtered_data)

            features = filtered_data.drop(columns=['평균가격(원)', '시점']).columns
            target = '평균가격(원)'
            X = filtered_data[features].values
            y = filtered_data[target].values

            X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
            model = train_model(X_train, y_train, X_val, y_val)

            joblib.dump(model, f'models/best_model_{품목명}.pkl')

            # 테스트 데이터 추론
            predictions = []
            for test_file in test_files:
                test_data, _ = preprocess_data(pd.read_csv(test_file), scaler=scaler)
                test_features = test_data[features].values
                predictions.extend(model.predict(test_features))

            품목별_predictions[품목명] = predictions

        except Exception as e:
            print(f"{품목명} 처리 중 오류 발생: {e}")

    sample_submission = pd.read_csv(output_file)
    for 품목명, predictions in 품목별_predictions.items():
        sample_submission[품목명] = predictions
    sample_submission.to_csv(output_file, index=False)


In [31]:
def main():
    # 데이터 경로 설정
    train_raw_file = "C:/Users/medici/pybasic/train.csv"
    train_산지공판장_file = "C:/Users/medici/pybasic/TRAIN_산지공판장_2018-2021.csv"
    train_전국도매_file = "C:/Users/medici/pybasic/TRAIN_전국도매_2018-2021.csv"
    test_files = [f"C:/Users/medici/pybasic/test/TEST_{i:02d}.csv" for i in range(25)]
    test_산지공판장_files = [f"C:/Users/medici/pybasic/test/meta/TEST_산지공판장_{i:02d}.csv" for i in range(25)]
    test_전국도매_files = [f"C:/Users/medici/pybasic/test/meta/TEST_전국도매_{i:02d}.csv" for i in range(25)]
    output_file = "C:/Users/medici/pybasic/baseline_submission_origin.csv"

    # 품목 리스트 및 거래단위
    품목_리스트 = ['감자', '건고추', '깐마늘(국산)', '대파', '무', '배추', '사과', '상추', '양파', '배']
    거래단위 = {'감자': 20, '깐마늘(국산)': 20, '대파': 1, '무': 20, '배추': 10, '사과': 3, '배': 5, '상추': 0.1, '양파': 1}

    # 타겟 조건
    conditions = {
        '감자': {
            'target': lambda df: (df['품종명'] == '감자 수미') & (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['감자'], '품종명': ['수미'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['감자'], '품종명': ['수미']}
        },
        '건고추': {
            'target': lambda df: (df['품종명'] == '화건') & (df['거래단위'] == '30 kg') & (df['등급'] == '상품'),
            '공판장': None, 
            '도매': None  
        },
        '깐마늘(국산)': {
            'target': lambda df: (df['거래단위'] == '20 kg') & (df['등급'] == '상품'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['마늘'], '품종명': ['깐마늘'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['마늘'], '품종명': ['깐마늘']}
        },
        '대파': {
            'target': lambda df: (df['품종명'] == '대파(일반)') & (df['거래단위'] == '1키로단') & (df['등급'] == '상'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['대파'], '품종명': ['대파(일반)'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['대파'], '품종명': ['대파(일반)']}
        },
        '무': {
            'target': lambda df: (df['거래단위'] == '20키로상자') & (df['등급'] == '상'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['무'], '품종명': ['기타무'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['무'], '품종명': ['무']}
        },
        '배추': {
            'target': lambda df: (df['거래단위'] == '10키로망대') & (df['등급'] == '상'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배추'], '품종명': ['쌈배추'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['배추'], '품종명': ['배추']}
        },
        '사과': {
            'target': lambda df: (df['품종명'].isin(['홍로', '후지'])) & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['사과'], '품종명': ['후지'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['사과'], '품종명': ['후지']}
        },
        '상추': {
            'target': lambda df: (df['품종명'] == '청') & (df['거래단위'] == '100 g') & (df['등급'] == '상품'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['상추'], '품종명': ['청상추'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['상추'], '품종명': ['청상추']}
        },
        '양파': {
            'target': lambda df: (df['품종명'] == '양파') & (df['거래단위'] == '1키로') & (df['등급'] == '상'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['양파'], '품종명': ['기타양파'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['양파'], '품종명': ['양파(일반)']}
        },
        '배': {
            'target': lambda df: (df['품종명'] == '신고') & (df['거래단위'] == '10 개') & (df['등급'] == '상품'),
            '공판장': {'공판장명': ['*전국농협공판장'], '품목명': ['배'], '품종명': ['신고'], '등급명': ['상']},
            '도매': {'시장명': ['*전국도매시장'], '품목명': ['배'], '품종명': ['신고']}
        }
    }

    train_files = (train_raw_file, train_산지공판장_file, train_전국도매_file)
    process_workflow(품목_리스트, 거래단위, conditions, train_files, test_files, output_file)


if __name__ == "__main__":
    main()

품목 처리 중:   0%|          | 0/10 [00:00<?, ?it/s]C:\Users\medici\AppData\Local\Temp\ipykernel_25852\1141033359.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered_data['시점_날짜'] = pd.to_datetime(filtered_data['시점'], errors='coerce')


Fitting 3 folds for each of 8640 candidates, totalling 25920 fits


품목 처리 중:  10%|█         | 1/10 [00:05<00:52,  5.80s/it]

감자 처리 중 오류 발생: could not convert string to float: '감자'


품목 처리 중:  20%|██        | 2/10 [00:24<01:47, 13.48s/it]

...

양파 처리 중 오류 발생: could not convert string to float: '양파'


C:\Users\medici\AppData\Local\Temp\ipykernel_25852\1141033359.py:64: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  filtered_data['시점_날짜'] = pd.to_datetime(filtered_data['시점'], errors='coerce')


Fitting 3 folds for each of 8640 candidates, totalling 25920 fits


품목 처리 중: 100%|██████████| 10/10 [02:04<00:00, 12.42s/it]

배 처리 중 오류 발생: could not convert string to float: '배'
